In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import cv2
from keras.models import load_model
import face_recognition
import keras.backend as K

/anaconda3/envs/kerasv6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Load video as iterable

In [2]:
#from imutils.video import FileVideoStream
#from imutils.video import FPS
#import numpy as np
#import argparse
#import imutils
#import time
#import cv2

https://www.pyimagesearch.com/2017/02/06/faster-video-file-fps-with-cv2-videocapture-and-opencv/



In [3]:
# Path video to analyze
path = './Videos/sad_affleck.mp4'


In [4]:
class VideoBuddy:
    def __init__(self, path, fps):
        # Create video instance
        self.stream = cv2.VideoCapture(path)
        # fps is the downsampled fps to apply Yolo
        self.fps = fps
        # Set threaded fps (Adrian R)
        self.stopped = False
        self.Q = Queue(maxsize=128)
        
    def start(self):
        thread = Thread(target=self.update, args=())
        thread.daemon = True
        thread.start()
        return self
        
    def update(self):
        while True:
            
            if self.stopped:
                return
            
            if not self.Q.full():
                ret, frame = self.stream.read()
                
            if not ret:
                self.stop()
                return
            self.Q.put(frame)
    
    def read(self):
        return self.Q.get()
    
    def more(self):
        return self.Q.qsize() > 0
    
    def stop(self):
        self.stopped = True

In [5]:
# Create the instance
vid = VideoBuddy(path, 120)

# Generate the thread
vid.start()

NameError: name 'Queue' is not defined

In [32]:
while vid.more():
    # grab the frame from the threaded video file stream, resize
    # it, and convert it to grayscale (while still retaining 3
    # channels)
    frame = vid.read()
    frame = imutils.resize(frame, width=450)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame = np.dstack([frame, frame, frame])
 
    # display the size of the queue on the frame
    cv2.putText(frame, "Queue Size: {}".format(fvs.Q.qsize()),
        (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)	
 
    # show the frame and update the FPS counter
    cv2.imshow("Frame", frame)
    cv2.waitKey(1)
    vid.update()

NotImplementedError: 

In [22]:
from yolo_utils import read_classes, read_anchors, generate_colors, preprocess_image, draw_boxes
from yad2k.models.keras_yolo import yolo_head, yolo_eval

from wide_resnet import WideResNet
from keras.utils.data_utils import get_file
from keras.models import load_model

#### Fast threaded implementation does not work on MacOS

In [106]:

class VideoBuddy:
    def __init__(self, path, fps=120, batch_size=128, verbosity=True, threshold=0.3):
        # Create video instance
        self.stream = cv2.VideoCapture(path)
        # fps is the downsampled fps to apply Yolo
        self.fps = fps
        self.stopped = False
        self.image_counter = 0
        self.batch_size = batch_size
        self.batch_counter = 0
        self.verbosity = verbosity
        self.threshold = threshold
    
    def set_age_gender_model(self):
        # load Age and gender model and weights
        img_size = 64
        depth = 16
        k = 8
        weight_file = 'pretrained_models/weights.18-4.06.hdf5'
        self.ag_model = WideResNet(img_size, depth=depth, k=k)()
        self.ag_model.load_weights(weight_file)

    def set_emotion_model(self):    
        # Load 
        emotion_model_path = 'pretrained_models/fer2013_mini_XCEPTION.102-0.66.hdf5'
        self.emotion_model = load_model(emotion_model_path)
        
    def init_yolo(self):
        # Init Yolo configuration
        self.class_names = read_classes("model_data/coco_classes.txt")
        self.anchors = read_anchors("model_data/yolo_anchors.txt")
        #Load the pretrained model
        self.yolo = load_model("model_data/yolo.h5")
        #Convert final layer features to bounding box parameters
        self.yolo_outputs = yolo_head(self.yolo.output, self.anchors, len(self.class_names))
        width = np.array(416, dtype=float)
        height = np.array(416, dtype=float)
        self.image_shape = (width, height)
        self.boxes, self.scores, self.classes = yolo_eval(self.yolo_outputs, self.image_shape)
        self.sess = K.get_session()
        return
        
    def read(self):
        # Check if the video is no at the end
        if self.stopped:
            return
        # get current frame
        success, frame = self.stream.read()
        # Check if is not the end of the video
        if not success:
            self.stopped = True
            print('Video Succesfully Analysed. {} images processed'.format(self.image_counter))
            return
        # Update image counter
        self.image_counter = self.image_counter + 1
        return frame
    
    def batch_gen(self):
        # Read and generate a single batch
        self.batch = [self.read() for index in range(self.batch_size)]
        return np.array(self.batch)
    
    def analyze(self):
        # Generate batchs until the end of the video
        self.yolo_predictions = []
        
        while not self.stopped:
            # Generate one batch
            self.current_batch = self.batch_gen()
            #self.predictions = self.object_detection()
            self.predictions = self.face_detection()
        
            self.yolo_predictions.append(self.predictions)    
            
            # Show some info
            if self.verbosity and not self.stopped:            
                self.batch_counter = self.batch_counter + 1
                print('Analyzing batch {}. Elapsed time ='.format(self.batch_counter))
                
        return self.yolo_predictions
    
    def object_detection(self):
        # Downsample batch to approx 1 image per second
        self.downsampled_batch = self.current_batch[::24]
        # Check is there is not empty elements in the list
        for frame in self.downsampled_batch:
            if frame is None:
                self.stopped = True
        # Resize images in batch, convert to numpy array and normalize
        if not self.stopped:
            self.downsampled_batch = [cv2.resize(frame, (416,416)) for frame in self.downsampled_batch]
            self.downsampled_batch = np.array(self.downsampled_batch, dtype='float32')
            self.downsampled_batch /= 255.
            # Call Yolo predictions with the dowsampled batch
            self.out_scores, self.out_boxes, self.out_classes = self.sess.run([self.scores, self.boxes, self.classes],
                                                    feed_dict= {self.yolo.input:self.downsampled_batch,
                                                                K.learning_phase(): 0})
        return self.out_classes
    
    def face_detection(self):
        # Convert frame to small frame
        self.total_faces = []
        self.total_emotions = []
        self.total_age = []
        self.total_gender = []
        self.downsampled_batch = self.current_batch[::24]
        # Check is there is not empty elements in the list
        for frame in self.downsampled_batch:
            if frame is None:
                self.stopped = True
                
        if not self.stopped:
            self.downsampled_batch = [cv2.resize(frame, (0, 0), fx=0.25, fy=0.25) for frame in self.downsampled_batch]
            self.downsampled_batch = [frame[:, :, ::-1] for frame in self.downsampled_batch]
            self.face_locations = [face_recognition.face_locations(rgb_frame, model='hog',
                                                               number_of_times_to_upsample=3) for rgb_frame in self.downsampled_batch]    

            self.emotion_detection()
        return
    
    def emotion_detection(self):
        
        self.face_emotions = []
        for face in self.face_locations:
            print(len(face))
            # Keep the number of faces detected on the frames
            self.total_faces.append(len(self.face_locations))
        
            print(len(self.face_locations))
            # Get face coordinates
            coord = []
            for element in face:
                top, right, bottom, left = element
                coord.append([top, right, bottom, left])
                
            # Crop image
            face_image = []
            for index, frame in enumerate(self.downsampled_batch):
                top, right, bottom, left = coord[index]
                face_image.append(frame[top:bottom, left:right])
                
            #face_image = [frame[top:bottom, left:right] for frame in self.rgb_small_frames]
            
            # Pre-process image
            color_img, gray_img = pre_processing(face_image)

            # Predict emotion and gender
            predicted_emotion = self.emotion_model.predict(gray_img)
            #print(predicted_emotion)
            # predict ages and genders of the detected faces
            results = self.ag_model.predict(color_img)
            predicted_gender = [result[0] for result in results]
            age = np.arange(0, 101).reshape(101, 1)
            predicted_age = [result[1].dot(age).flatten() for result in results]

            # Keep local predictions for final analysis
            self.total_emotions.append(np.argmax(predicted_emotion))
            self.total_age.append(int(predicted_age))
            self.total_gender.append(np.argmax(predicted_gender))

            # Decode outputs
            age_prediction = [str(age)[1:3] for age in predicted_age]
            
            #age_prediction =  #emotion_decoder(emotion_output, 0.30)
            #gender_prediction = gender_decoder(predicted_gender)
            gender_prediction = [gender_decoder(gender) for gender in predicted_gender]

            #emotion_prediction = emotion_decoder(predicted_emotion)
            emotion_prediction = [emotion_decoder(emotion) for emotion in predicted_emotion]

            # Create text to add on the rectangle
            prediction = emotion_prediction + '\n' + gender_prediction + '\n' + age_prediction

            # Add predictions to rectangle
            self.face_emotions.append(prediction)
        
        return self.face_emotions
    
    def annotate_frame(self):
        for (top, right, bottom, left), emotion in zip(self.face_locations, self.face_emotions):
            # Scale back up face locations since the frame we detected in was scaled to 1/4 size
            top *= 4
            right *= 4
            bottom *= 4
            left *= 4

            # Create a copy of the frame as overlay
            # CREATE ANOTHER BATCH VERSION
            overlay = frame.copy()

            # Draw a box around the face
            cv2.rectangle(overlay, (left, top), (right, bottom), (180, 0, 0), 2) #Blue

            # Plot predictions
            font = cv2.FONT_HERSHEY_DUPLEX
            y0, dy = 0, 30
            for i, line in enumerate(emotion.split('\n')):
                y = y0 + i*dy
                cv2.putText(overlay, line, (left + 6, bottom - 6 + y), font, 1.0, (255, 255, 255), 1)
            opacity = 0.5
            cv2.addWeighted(overlay, opacity, frame, 1 - opacity, 0, frame)
    
    def gender_decoder(self):
        self.net_output = self.predicted_gender
        if np.max(self.net_output) > self.threshold:
            probability = str(np.max(self.net_output))
            if np.argmax(self.net_output) == 0:
                prediction = 'female'
            else:
                prediction = 'male'
        else:
            prediction = ''
            probability = ''

        self.gender_pred = prediction + ' ' + probability[:5] 
        
        return self.gender_pred
    
    
    def emotion_decoder(self):
    # Check if probability is higher than threshold
        if np.max(self.net_output) >= self.threshold:
            probability = str(np.max(self.net_output))
            if (np.argmax(self.net_output) == 0):
                prediction = 'Angry'
            elif (np.argmax(self.net_output) == 1):
                prediction = 'Disgust'
            elif (np.argmax(self.net_output) == 2):
                prediction = 'Fear'
            elif (np.argmax(self.net_output) == 3):
                prediction = 'Happy'
            elif (np.argmax(self.net_output) == 4):
                prediction = 'Sad'
            elif (np.argmax(self.net_output) == 5):
                prediction = 'Surprise'
            else:
                prediction = 'Neutral'
        else:
            # Do not label the box if you are not sure
            prediction = ''
            probability = ''
        emotion_pred = prediction + ' ' + probability[:5]

        return self.emotion_pred
    

    def pre_processing(self):
        # Convert image to B&W
        gray_frames = [cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY) for frame in self.downsampled_batch]
        gray_frames = [normalize_input(frame) for frame in gray_frames]
        gray_frames = [cv2.resize(frame,(64,64), interpolation = cv2.INTER_CUBIC) for frame in gray_frames]
        self.gray_frames = np.reshape(gray_frames,(-1,64,64,1))

        # Color image
        color_frames = [cv2.resize(frame,(64,64), interpolation = cv2.INTER_CUBIC) 
                        for frame in self.downsampled_batch]
        
        self.color_frames = np.reshape(color_frames,(-1, 64,64,3))
        
        return self.color_frames, self.gray_frames
    
    def normalize_input(self, v2=True):
        x = self.gray_frames.astype('float32')
        x = x / 255.0
        if v2:
            x = x - 0.5
            self.normalized_batch = x * 2.0
        return self.normalized_batch
    
    def reset_counter(self):
        self.image_counter = 0
        return self.image_counter
    
    def filter_artifacts(self):
        return
    
    def find(self):
        return
    
    def generate_video(self):
        return
    
    def emotion_recognition(self):
        return
    

In [107]:
vid = VideoBuddy(path)

In [108]:
#vid.init_yolo()

In [109]:
# add path
vid.set_age_gender_model()

In [110]:
# add path
vid.set_emotion_model()

/anaconda3/envs/kerasv6/lib/python3.6/site-packages/keras/models.py:318: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [111]:
pred = vid.analyze()

0
6


IndexError: list index out of range

In [110]:
pred

[array([0, 0]),
 array([0, 0]),
 array([0, 0]),
 array([], dtype=int64),
 array([], dtype=int64),
 array([0, 0]),
 array([0, 0]),
 array([ 0,  0, 65]),
 array([0, 0]),
 array([0, 0]),
 array([0, 0]),
 array([0, 0]),
 array([0, 0]),
 array([0, 0]),
 array([0, 0]),
 array([0, 0]),
 array([0, 0])]